In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
import os

prefix = 'train_data'
ecr_repository_name = 'neural_network_spam_classifier'
role = get_execution_role()
account_id = role.split(':')[4]
region = boto3.Session().region_name
bucket = 'sagemaker-neural-net'
sagemaker_session = sagemaker.session.Session(default_bucket=bucket)

print(account_id)
print(region)
print(role)
print(bucket)

540748271236
us-east-1
arn:aws:iam::540748271236:role/service-role/AmazonSageMaker-ExecutionRole-20200831T143512
sagemaker-neural-net


In [11]:
! pygmentize ./container/build.sh

%%sh

# The name of our algorithm
algorithm_name=neural_network_spam_classifier

#cd container

chmod +x source_dir/train.py

#aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.2.0-cpu-py37-ubuntu18.04
#
#docker pull 763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.2.0-cpu-py37-ubuntu18.04

account=$(aws sts get-caller-identity --query Account --output text)

region=us-east-1

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr --region us-east-1 describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr --region us-east-1 create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region us-east-1 | docker login --username

In [12]:
%%capture
! ./scripts/build_and_push.sh 540748271236 us-east-1 sagemaker-training-containers/script-mode-container

In [13]:
container_image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/{2}:version4'.format(account_id, region, ecr_repository_name)
print(container_image_uri)

540748271236.dkr.ecr.us-east-1.amazonaws.com/neural_network_spam_classifier:version4


In [32]:
print(sagemaker_session.upload_data(os.path.join(os.getcwd(),'container','source_dir','mids.csv'), bucket, prefix + '/train'))
print(sagemaker_session.upload_data(os.path.join(os.getcwd(),'container','source_dir','mids.csv'), bucket, prefix + '/validation'))

s3://sagemaker-neural-net/train_data/train/mids.csv
s3://sagemaker-neural-net/train_data/validation/mids.csv


In [14]:
import sagemaker
import json

# JSON encode hyperparameters
def json_encode_hyperparameters(hyperparameters):
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

hyperparameters = json_encode_hyperparameters({
    "learning-rate": 0.02,
    "epochs": 100})

est = sagemaker.estimator.Estimator(container_image_uri,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='local', # we use local mode
                                    #train_instance_type='ml.m5.xlarge',
                                    base_job_name='neural_net_spam_classifier',
                                    output_path='s3://sagemaker-neural-net/neural-network/',
                                    hyperparameters=hyperparameters)

train_config = sagemaker.session.s3_input('s3://{0}/{1}/train/'.format(bucket, prefix), content_type='text/csv')
val_config = sagemaker.session.s3_input('s3://{0}/{1}/validation/'.format(bucket, prefix), content_type='text/csv')

est.fit({'train': train_config, 'validation': val_config })

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Creating tmph8snsvvd_algo-1-bbkjw_1 ... 
Attaching to tmph8snsvvd_algo-1-bbkjw_12mdone
algo-1-bbkjw_1  | 2020-09-24 08:40:57,367 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
algo-1-bbkjw_1  | /usr/bin/python3 -m pip install -r requirements.txt
algo-1-bbkjw_1  | Requirement already satisfied: boto3==1.15.3 in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 1)) (1.15.3)
algo-1-bbkjw_1  | Requirement already satisfied: h5py==2.10.0 in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 2)) (2.10.0)
algo-1-bbkjw_1  | Requirement already satisfied: html2text==2020.1.16 in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 3)) (2020.1.16)
algo-1-bbkjw_1  | Requirement already satisfied: nltk==3.5 in /usr/local/lib/python3.6/dist-packages (from -r requirements.txt (line 4)) (3.5)
algo-1-bbkjw_1  | Requirement already satisfied: numpy==1.16.0 in /usr/local/lib/python3.6/dist-packages (from -r re

In [49]:
import json
from time import gmtime, strftime

tuning_job_name = 'SpamClassifierNN-HPTuning'

print(tuning_job_name)

tuning_job_config = {
    "ParameterRanges": {
      "CategoricalParameterRanges": [],
      "ContinuousParameterRanges": [
        {
          "MaxValue": "0.001",
          "MinValue": "0.0001",
          "Name": "learning-rate",          
        }
      ],
      "IntegerParameterRanges": []
    },
    "ResourceLimits": {
      "MaxNumberOfTrainingJobs": 9,
      "MaxParallelTrainingJobs": 3
    },
    "Strategy": "Bayesian",
    "HyperParameterTuningJobObjective": {
      "MetricName": "loss",
      "Type": "Minimize"
    }
  }

SpamClassifierNN-HPTuning


In [50]:
training_image = container_image_uri


training_job_definition = {
    "AlgorithmSpecification": {
      "MetricDefinitions": [
        {
          "Name": "loss",
          "Regex": "loss: ([0-9\\.]+)"
        }
      ],
      "TrainingImage": training_image,
      "TrainingInputMode": "File"
    },
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": 's3://{0}/{1}/train/'.format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": 's3://{0}/{1}/validation/'.format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated"
                }
            },            
            "CompressionType": "None",
            "RecordWrapperType": "None"            
        }
    ],
    "OutputDataConfig": {
      "S3OutputPath": "s3://{0}/{1}/output".format(bucket,prefix)
    },
    "ResourceConfig": {
      "InstanceCount": 1,
      "InstanceType": "ml.m5.xlarge",
      "VolumeSizeInGB": 5
    },
    "RoleArn": role,
    "StoppingCondition": {
      "MaxRuntimeInSeconds": 3600
    }
}

In [51]:
smclient = boto3.client('sagemaker')
smclient.create_hyper_parameter_tuning_job(HyperParameterTuningJobName = tuning_job_name,
                                               HyperParameterTuningJobConfig = tuning_job_config,
                                               TrainingJobDefinition = training_job_definition)

{'HyperParameterTuningJobArn': 'arn:aws:sagemaker:us-east-1:540748271236:hyper-parameter-tuning-job/spamclassifiernn-hptuning',
 'ResponseMetadata': {'RequestId': 'dff79e1e-2feb-476c-b9bf-2509deac18d2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dff79e1e-2feb-476c-b9bf-2509deac18d2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '126',
   'date': 'Thu, 24 Sep 2020 11:04:58 GMT'},
  'RetryAttempts': 0}}

In [54]:
smclient.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName = tuning_job_name)['HyperParameterTuningJobStatus']

'Completed'

2020-09-24
24-11-04
